# disRNN Result Access — Example Usage

This notebook demonstrates how to use `aind-disrnn-result-access` to query
W&B run metadata and download training artifacts.

**Prerequisites:**
- Install the package: `uv sync` or `pip install -e .`
- Authenticate with W&B: `wandb login`

## 1. Initialize the Client

In [ ]:
from aind_disrnn_result_access import WandbClient

client = WandbClient()  # defaults to entity="AIND-disRNN"

## 2. List Available Projects

In [ ]:
projects = client.get_projects()
print("Available projects:")
for p in projects:
    print(f"  - {p}")

## 3. Browse Runs

List runs in a project. You can filter and sort them.

In [ ]:
# Pick a project (change this to whichever project you want to explore)
PROJECT = "han_mice_disrnn"

In [ ]:
runs = client.get_runs(project=PROJECT)
print(f"Found {len(runs)} runs in '{PROJECT}'")
for run in runs[:5]:  # show first 5
    print(f"  [{run.id}] {run.name} — state={run.state}")

### Filter runs

Use [MongoDB-style queries](https://docs.wandb.ai/guides/runs/filter-runs) to filter runs.

In [ ]:
finished_runs = client.get_runs(
    project=PROJECT,
    filters={"state": "finished"},
)
print(f"Found {len(finished_runs)} finished runs")

## 4. Inspect a Single Run

Get detailed metadata for a specific run.

In [ ]:
# Use the first run from our list (or replace with a known run ID)
if runs:
    run = client.get_run(runs[0].id, project=PROJECT)
    print(f"Run: {run.name} ({run.id})")
    print(f"State: {run.state}")
    print(f"Tags: {run.tags}")
    print(f"Created: {run.created_at}")
    print(f"URL: {run.url}")
else:
    print("No runs found — adjust PROJECT above.")

### Run config (training hyperparameters)

In [ ]:
if runs:
    print("Config:")
    for key, value in run.config.items():
        print(f"  {key}: {value}")

### Run summary (final metrics)

In [ ]:
if runs:
    print("Summary metrics:")
    for key, value in run.summary.items():
        print(f"  {key}: {value}")

## 5. Download Artifacts

Download training output artifacts (model parameters, plots, CSVs) for a run.

In [ ]:
if runs:
    artifacts = client.download_artifact(
        runs[0].id,
        project=PROJECT,
        output_dir="./downloaded_artifacts",
    )
    for art in artifacts:
        print(f"Artifact: {art.name} (type={art.type}, version={art.version})")
        print(f"  Downloaded to: {art.download_path}")
        print(f"  Files: {art.files}")

### Batch download for multiple runs

In [ ]:
if len(runs) >= 2:
    run_ids = [r.id for r in runs[:2]]
    all_artifacts = client.download_artifacts(
        run_ids,
        project=PROJECT,
        output_dir="./downloaded_artifacts",
    )
    for rid, arts in all_artifacts.items():
        print(f"Run {rid}: {len(arts)} artifact(s) downloaded")

## 6. Explore Downloaded Files

After downloading, artifacts are available as local files.

In [ ]:
import json
from pathlib import Path

if runs and artifacts:
    artifact_dir = artifacts[0].download_path
    print(f"Contents of {artifact_dir}:")
    for f in sorted(Path(artifact_dir).rglob("*")):
        if f.is_file():
            print(f"  {f.relative_to(artifact_dir)}")

    # Example: load params.json if it exists
    params_file = artifact_dir / "params.json"
    if params_file.exists():
        with open(params_file) as fh:
            params = json.load(fh)
        print(f"\nLoaded params.json with {len(params)} keys")
        for k, v in list(params.items())[:5]:
            print(f"  {k}: {v}")